In [ ]:
!pip install transformers torch pandas accelerate quanto flash_attn

In [ ]:
from transformers import pipeline
import torch
import pandas as pd
import glob
import csv

In [ ]:
dfs = []

def read_file(file):
    try:
        return open(directory+file+".txt", "r").read()
    except:
        return None

for info_file in glob.glob("RusLit/**/info.csv", recursive=True):
    try:
        directory = info_file[:-8]
        info = pd.read_csv(info_file)
        info["author"] = directory.split("/")[2]
        info["genre"] = directory.split("/")[1]
        info["text"] = info["name"].map(read_file)
        dfs.append(info)
    except:
        pass

df = pd.concat(dfs, ignore_index=True).dropna()
df_small = df[df["text"].str.len() < 16384]
df_small

In [38]:
pipe = pipeline("text-generation", model="cognitivecomputations/dolphin-2.9-llama3-8b", model_kwargs={"torch_dtype": torch.bfloat16, "attn_implementation": "flash_attention_2"}, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
generated_texts = pipe(df_small["text"].map(lambda x: x + "List all potential genres in a comma seperated list:").to_list(), max_new_tokens=20)

In [ ]:
genres = [generated_text[0]["generated_text"].split("List all potential genres in a comma seperated list:")[1].split(", ") for generated_text in generated_texts]